In [1]:
import pandas as pd
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
#df = pd.read_csv('/content/drive/My Drive/CIS-5190-Project/Code/raw/wsj_full_articles_pt1.csv', on_bad_lines='warn')


Skipping line 496: expected 6 fields, saw 11



In [3]:
df = pd.read_csv('/content/drive/My Drive/CIS-5190-Project/Code/raw/wsj_full_articles_pt2.csv', on_bad_lines='warn')


In [4]:
def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^\w\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [7]:
df

,date,headline,url,theme,timestamp,text,clean_text
0,2013-01-01,lack of grand bargain adds complications,http://online.wsj.com/article/SB10001424127887...,politics,11:39 PM ET,"For President Barack Obama, the new year was s...","For President Barack Obama, the new year was s..."
1,2013-01-01,some executives hoped for broader reach,http://online.wsj.com/article/SB10001424127887...,politics,11:37 PM ET,Some corporate leaders are relieved lawmakers ...,Some corporate leaders are relieved lawmakers ...
2,2013-01-01,congress averts dairy cliff,http://online.wsj.com/article/SB10001424127887...,politics,11:36 PM ET,WASHINGTON—The fiscal-cliff bill that cleared ...,WASHINGTONThe fiscalcliff bill that cleared Co...
3,2013-01-01,renewableenergy tax breaks pass despite headwind,http://online.wsj.com/article/SB10001424127887...,politics,11:31 PM ET,WASHINGTON—The fiscal-cliff deal approved in t...,WASHINGTONThe fiscalcliff deal approved in the...
4,2013-01-01,boehner shepherds deal through house,http://online.wsj.com/article/SB10001424127887...,politics,11:28 PM ET,"House Speaker John Boehner, who for two years ...","House Speaker John Boehner, who for two years ..."
...,...,...,...,...,...,...,...
83959,2023-12-31,investors have cut chinas internet giants down...,https://www.wsj.com/finance/stocks/investors-h...,markets,12:20 AM ET,HONG KONG—Investors in China’s biggest interne...,HONG KONGInvestors in Chinas biggest internet ...
83960,2024-01-01,optimism abounds on wall street this new year,https://www.wsj.com/finance/optimism-abounds-w...,markets,5:30 AM ET,Wall Street is feeling sunny about the stock m...,Wall Street is feeling sunny about the stock m...
83961,2024-01-01,kim jong uns 2024 wish list more nuclear bombs...,https://www.wsj.com/world/asia/kim-jong-uns-20...,asia,2:43 AM ET,"SEOUL—Throughout 2023, North Korean leader Kim...","SEOULThroughout 2023, North Korean leader Kim ..."
83962,2024-01-01,chinas xi is resurrecting maos continuous revo...,https://www.wsj.com/world/china/chinas-xi-is-r...,china,12:08 AM ET,Chinese rulers have long used campaigns agains...,Chinese rulers have long used campaigns agains...


In [9]:
df['clean_text'] = df['text'].apply(clean_text)


In [10]:
df.shape[0]

83964

In [11]:
keywords = [
    "China", "Chinese", "Yuan", "¥", "Renminbi", "CNY", "RMB", "Beijing", "Shanghai", "Shenzhen", "People's Currency", "元", "圆",
    "Hong Kong", "CCP", "Xi Jinping", "Belt and Road", "Silk Road",
    "Taiwan", "Trade War", "Sino",
    "PBOC", "Alibaba", "Tencent", "Baidu", "Weibo", "Huawei", "ByteDance", "ZTE", "Sinopec", "Haier", "Geely", "Xiaomi", "Hang Seng",
    "Mandarin", "Cantonese", "Sichuan", "Fujian",
    "Guangdong", "One Belt One Road", "AIIB"
]

pattern = '|'.join(keywords)

In [12]:
filtered_articles = df[df['clean_text'].str.contains(pattern, case=False, na=False)]

In [13]:
filtered_articles.shape[0]

15062

In [14]:

tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [15]:

def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = softmax(outputs.logits, dim=1)
    return probs.detach().numpy()


In [16]:

filtered_articles['sentiment_probs'] = filtered_articles['clean_text'].apply(sentiment_analysis)
filtered_articles['sentiment'] = filtered_articles['sentiment_probs'].apply(lambda x: ['negative', 'neutral', 'positive'][x.argmax()])


<ipython-input-16-165520b49c71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_articles['sentiment_probs'] = filtered_articles['clean_text'].apply(sentiment_analysis)
<ipython-input-16-165520b49c71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_articles['sentiment'] = filtered_articles['sentiment_probs'].apply(lambda x: ['negative', 'neutral', 'positive'][x.argmax()])


In [1]:
filtered_articles.to_csv('/content/drive/My Drive/CIS-5190-Project/Code/cleaned/wsj_full_sent_pt2.csv', index=False)
#filtered_articles.to_csv('/content/drive/My Drive/CIS-5190-Project/Code/cleaned/wsj_full_sent_pt2.csv', index=False)

SyntaxError: invalid syntax (<ipython-input-1-b3c9fd4b1470>, line 1)

In [19]:
filtered_articles

,date,headline,url,theme,timestamp,text,clean_text,sentiment_probs,sentiment
11,2013-01-01,china manufacturing grows modestly,http://online.wsj.com/article/SB10001424127887...,china news,3:12 PM ET,BEIJING—Manufacturing activity in China expand...,BEIJINGManufacturing activity in China expande...,"[[0.94845766, 0.003421064, 0.048121218]]",negative
12,2013-01-01,global brewers face challenges in india,http://online.wsj.com/article/SB10001424127887...,asia business,12:24 PM ET,Diageo PLC's November acquisition of a majorit...,Diageo PLCs November acquisition of a majority...,"[[0.95059645, 0.017474383, 0.031929117]]",negative
15,2013-01-01,ny times reporter forced to leave china,http://online.wsj.com/article/SB10001424127887...,china news,8:44 AM ET,BEIJING—A New York Times reporter was forced t...,BEIJINGA New York Times reporter was forced to...,"[[0.99990034, 3.904676e-07, 9.9330704e-05]]",negative
22,2013-01-02,new zealand dollar higher late but pares some ...,http://online.wsj.com/article/SB10001424127887...,dj fx trader,11:02 PM ET,WELLINGTON--The New Zealand dollar was trading...,WELLINGTONThe New Zealand dollar was trading h...,"[[0.6458235, 0.118888445, 0.23528811]]",negative
23,2013-01-02,starbucks to open first store in vietnam,http://online.wsj.com/article/SB10001424127887...,asia business,10:54 PM ET,Starbucks Corp. outlined the details of its pl...,Starbucks Corp. outlined the details of its pl...,"[[0.9999851, 1.4730984e-05, 7.99661e-08]]",negative
...,...,...,...,...,...,...,...,...,...
83956,2023-12-29,china confronts a new political reality in tai...,https://www.wsj.com/world/asia/china-confronts...,asia,12:08 AM ET,TAIPEI—A drawing of Taiwan at the presidential...,TAIPEIA drawing of Taiwan at the presidential ...,"[[0.52263516, 0.00021626416, 0.4771486]]",negative
83957,2023-12-30,taiwan says chinese lipsyncing probe aims to p...,https://www.wsj.com/world/asia/taiwan-says-chi...,asia,8:06 AM ET,TAIPEI—A Chinese investigation into purported ...,TAIPEIA Chinese investigation into purported l...,"[[0.6691813, 4.3364056e-05, 0.33077526]]",negative
83958,2023-12-31,chinas xi warns of economic winds and rains as...,https://www.wsj.com/world/china/china-ends-und...,china,7:46 AM ET,Chinese leader Xi Jinping urged his countrymen...,Chinese leader Xi Jinping urged his countrymen...,"[[2.340615e-06, 6.641828e-06, 0.99999106]]",positive
83959,2023-12-31,investors have cut chinas internet giants down...,https://www.wsj.com/finance/stocks/investors-h...,markets,12:20 AM ET,HONG KONG—Investors in China’s biggest interne...,HONG KONGInvestors in Chinas biggest internet ...,"[[5.597167e-06, 3.7058307e-05, 0.9999573]]",positive


In [18]:

print(filtered_articles[['date', 'clean_headline', 'sentiment', 'sentiment_probs']])

KeyError: "['clean_headline'] not in index"